In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

**Dataset**

In [13]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


**Architecture**

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

**Hyperparameters**

In [15]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

**Loss Function**

In [16]:
loss_fn = nn.CrossEntropyLoss()

**Optimizer**

In [17]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**Train Model**

In [18]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300874  [   64/60000]
loss: 2.280496  [ 6464/60000]
loss: 2.258363  [12864/60000]
loss: 2.251217  [19264/60000]
loss: 2.225681  [25664/60000]
loss: 2.199810  [32064/60000]
loss: 2.201290  [38464/60000]
loss: 2.169232  [44864/60000]
loss: 2.166317  [51264/60000]
loss: 2.129716  [57664/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 2.123316 

Epoch 2
-------------------------------
loss: 2.136759  [   64/60000]
loss: 2.124194  [ 6464/60000]
loss: 2.063563  [12864/60000]
loss: 2.081148  [19264/60000]
loss: 2.016143  [25664/60000]
loss: 1.959573  [32064/60000]
loss: 1.980693  [38464/60000]
loss: 1.905674  [44864/60000]
loss: 1.909801  [51264/60000]
loss: 1.826885  [57664/60000]
Test Error: 
 Accuracy: 60.3%, Avg loss: 1.830790 

Epoch 3
-------------------------------
loss: 1.867406  [   64/60000]
loss: 1.835014  [ 6464/60000]
loss: 1.719348  [12864/60000]
loss: 1.760981  [19264/60000]
loss: 1.632597  [25664/60000]
loss: 1.599447  [32064/600

In [20]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0150, -0.0214, -0.0056,  ..., -0.0023, -0.0274, -0.0089],
                      [-0.0219,  0.0099,  0.0095,  ...,  0.0182, -0.0191, -0.0221],
                      [ 0.0040,  0.0267, -0.0062,  ...,  0.0355,  0.0161, -0.0099],
                      ...,
                      [ 0.0198, -0.0096, -0.0252,  ...,  0.0021,  0.0077,  0.0293],
                      [ 0.0319,  0.0197, -0.0209,  ...,  0.0007,  0.0206,  0.0315],
                      [ 0.0331,  0.0182,  0.0047,  ...,  0.0210,  0.0199,  0.0121]])),
             ('linear_relu_stack.0.bias',
              tensor([-0.0054,  0.0400, -0.0267,  0.0154, -0.0265,  0.0216, -0.0236, -0.0296,
                       0.0325, -0.0065,  0.0045,  0.0267, -0.0334,  0.0028,  0.0112, -0.0082,
                       0.0107,  0.0194, -0.0085, -0.0215,  0.0064,  0.0094, -0.0229,  0.0367,
                       0.0419, -0.0332,  0.0072,  0.0259, -0.0314, -0.0245,  0.0076, -0.0149,
    

**Save model**

In [21]:
torch.save(model.state_dict(), 'basic_nn_fashion_mnist.pth')

**Load Mode**

In [22]:
model.load_state_dict(torch.load('basic_nn_fashion_mnist.pth', weights_only=True))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

**Save Architecture + Weights**

In [23]:
torch.save(model, 'basic_nn_fashion_mnist_architecture.pth')

**Load Architecture + Weights**

In [27]:
model = torch.load('basic_nn_fashion_mnist_architecture.pth', weights_only=False)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)